# 81. DF-LSH独立実装と評価

## 目的
- DF-LSH (Double Filters LSH) の2つの中核概念をITQとは独立に実装
- フィルタ1: DAHBF (Data-Aware Band Indexing) - PCA射影ベースのバンドインデックス
- フィルタ2: 幾何学的probe順序 - 射影値の確信度に基づくmulti-probe
- ITQ-LSHとのhead-to-head比較（同じデータ・同じ評価基準）

## 比較
| 手法 | 射影 | 量子化 | 検索方式 |
|------|------|--------|----------|
| DF-LSH | PCA（回転なし） | 符号量子化 | バンドインデックス + 幾何probe |
| ITQ-LSH | PCA + ITQ回転 | 符号量子化 | Hamming距離ソート |

## 0. セットアップ

In [1]:
import sys
import numpy as np
import time
from pathlib import Path
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, '../src')
from itq_lsh import ITQLSH, hamming_distance_batch
from dflsh import DFLSH, build_band_index, band_filter, confidence_multiprobe

DATA_DIR = Path('../data')
np.random.seed(42)

N_QUERIES = 100
TOP_K = 10

print('DF-LSH Standalone Evaluation')
print(f'  Queries: {N_QUERIES}, Top-K: {TOP_K}')

DF-LSH Standalone Evaluation
  Queries: 100, Top-K: 10


## 1. データロード

In [2]:
# 英語データ
en_embeddings = np.load(DATA_DIR / '10k_e5_base_en_embeddings.npy')
en_hashes = np.load(DATA_DIR / '10k_e5_base_en_hashes_128bits.npy')
print(f'English: embeddings={en_embeddings.shape}, hashes={en_hashes.shape}')

# 日本語データ
ja_embeddings = np.load(DATA_DIR / '10k_e5_base_ja_embeddings.npy')
ja_hashes = np.load(DATA_DIR / '10k_e5_base_ja_hashes_128bits.npy')
print(f'Japanese: embeddings={ja_embeddings.shape}, hashes={ja_hashes.shape}')

# MiniLMデータ
minilm_embeddings = np.load(DATA_DIR / '10k_minilm_embeddings.npy')
minilm_hashes = np.load(DATA_DIR / '10k_minilm_hashes_128bits.npy')
print(f'MiniLM:   embeddings={minilm_embeddings.shape}, hashes={minilm_hashes.shape}')

# ITQモデル
itq = ITQLSH.load(str(DATA_DIR / 'itq_e5_base_128bits.pkl'))
itq_minilm = ITQLSH.load(str(DATA_DIR / 'itq_minilm_128bits.pkl'))
print('ITQ models loaded.')

English: embeddings=(10000, 768), hashes=(10000, 128)
Japanese: embeddings=(10000, 768), hashes=(10000, 128)
MiniLM:   embeddings=(10000, 384), hashes=(10000, 128)
ITQ models loaded.


## 2. DF-LSH学習（PCA vs Random）

In [3]:
# PCA版 DF-LSH（英語）
print('='*60)
print('Training DF-LSH (PCA) on English data...')
dflsh_pca_en = DFLSH(n_projections=128, band_width=16, seed=42)
dflsh_pca_en.fit(en_embeddings, method='pca')

# Random版 DF-LSH（英語）
print('\nTraining DF-LSH (Random) on English data...')
dflsh_rand_en = DFLSH(n_projections=128, band_width=16, seed=42)
dflsh_rand_en.fit(en_embeddings, method='random')

# PCA版 DF-LSH（日本語）
print('\nTraining DF-LSH (PCA) on Japanese data...')
dflsh_pca_ja = DFLSH(n_projections=128, band_width=16, seed=42)
dflsh_pca_ja.fit(ja_embeddings, method='pca')

Training DF-LSH (PCA) on English data...
DFLSH fit (PCA): dim=768, projections=128, explained_variance=65.76%

Training DF-LSH (Random) on English data...
DFLSH fit (Random): dim=768, projections=128

Training DF-LSH (PCA) on Japanese data...


DFLSH fit (PCA): dim=768, projections=128, explained_variance=66.15%


In [4]:
# バイナリコード生成
print('Generating binary codes...')

# 英語
dflsh_pca_en_codes, dflsh_pca_en_proj = dflsh_pca_en.transform_with_confidence(en_embeddings)
dflsh_rand_en_codes, dflsh_rand_en_proj = dflsh_rand_en.transform_with_confidence(en_embeddings)
print(f'  PCA English codes: {dflsh_pca_en_codes.shape}, bit_balance: {dflsh_pca_en_codes.mean():.3f}')
print(f'  Random English codes: {dflsh_rand_en_codes.shape}, bit_balance: {dflsh_rand_en_codes.mean():.3f}')

# 日本語
dflsh_pca_ja_codes, dflsh_pca_ja_proj = dflsh_pca_ja.transform_with_confidence(ja_embeddings)
print(f'  PCA Japanese codes: {dflsh_pca_ja_codes.shape}, bit_balance: {dflsh_pca_ja_codes.mean():.3f}')

Generating binary codes...
  PCA English codes: (10000, 128), bit_balance: 0.501
  Random English codes: (10000, 128), bit_balance: 0.500
  PCA Japanese codes: (10000, 128), bit_balance: 0.500


## 3. バイナリコード品質分析

In [5]:
def evaluate_code_quality(embeddings, codes, label, n_queries=100):
    """バイナリコードのHamming-Cosine相関を評価"""
    rng = np.random.default_rng(42)
    query_indices = rng.choice(len(embeddings), n_queries, replace=False)
    
    all_hamming = []
    all_cosine = []
    
    for qi in query_indices:
        h_dists = hamming_distance_batch(codes[qi], codes).astype(float)
        c_sims = cosine_similarity(embeddings[qi:qi+1], embeddings)[0]
        mask = np.ones(len(embeddings), dtype=bool)
        mask[qi] = False
        all_hamming.extend(h_dists[mask])
        all_cosine.extend(c_sims[mask])
    
    corr, pval = spearmanr(all_hamming, all_cosine)
    print(f'{label}: Spearman={corr:.4f} (p={pval:.2e})')
    return corr

print('='*60)
print('Binary Code Quality (Hamming-Cosine Spearman Correlation)')
print('='*60)

print('\n--- English ---')
corr_itq_en = evaluate_code_quality(en_embeddings, en_hashes, 'ITQ-LSH')
corr_pca_en = evaluate_code_quality(en_embeddings, dflsh_pca_en_codes, 'DF-LSH (PCA)')
corr_rand_en = evaluate_code_quality(en_embeddings, dflsh_rand_en_codes, 'DF-LSH (Random)')

print('\n--- Japanese ---')
corr_itq_ja = evaluate_code_quality(ja_embeddings, ja_hashes, 'ITQ-LSH')
corr_pca_ja = evaluate_code_quality(ja_embeddings, dflsh_pca_ja_codes, 'DF-LSH (PCA)')

Binary Code Quality (Hamming-Cosine Spearman Correlation)

--- English ---


ITQ-LSH: Spearman=-0.4717 (p=0.00e+00)


DF-LSH (PCA): Spearman=-0.2278 (p=0.00e+00)


DF-LSH (Random): Spearman=-0.2505 (p=0.00e+00)

--- Japanese ---


ITQ-LSH: Spearman=-0.5283 (p=0.00e+00)


DF-LSH (PCA): Spearman=-0.2771 (p=0.00e+00)


## 4. バンドインデックス評価

In [6]:
def evaluate_band_filter(
    embeddings, codes, projections, band_width, 
    label, n_queries=100, top_k=10, max_probes_list=[0, 4, 8, 16]
):
    """バンドフィルタ+multi-probeの評価"""
    rng = np.random.default_rng(42)
    query_indices = rng.choice(len(embeddings), n_queries, replace=False)
    
    # バンドインデックス構築
    bi = build_band_index(codes, band_width)
    n_bands = codes.shape[1] // band_width
    
    results = []
    
    for max_probes in max_probes_list:
        filter_recalls = []
        candidate_counts = []
        recall_at_k = []
        times = []
        
        for qi in query_indices:
            # Ground truth
            cos_sims = cosine_similarity(embeddings[qi:qi+1], embeddings)[0]
            cos_sims[qi] = -1
            gt = set(np.argsort(cos_sims)[-top_k:])
            
            # バンドフィルタ + multi-probe
            start = time.time()
            if max_probes > 0:
                candidates = confidence_multiprobe(
                    codes[qi], projections[qi], bi, band_width,
                    max_probes=max_probes, order='confidence'
                )
            else:
                candidates = band_filter(codes[qi], bi, band_width, min_matches=1)
            query_time = time.time() - start
            times.append(query_time)
            
            candidate_set = set(candidates) - {qi}
            candidate_counts.append(len(candidate_set))
            filter_recalls.append(len(gt & candidate_set) / top_k)
            
            # Cosine rerankでのRecall@K
            if len(candidate_set) > 0:
                cand_arr = np.array(list(candidate_set))
                cand_cos = cosine_similarity(embeddings[qi:qi+1], embeddings[cand_arr])[0]
                top_in_cand = cand_arr[np.argsort(cand_cos)[-top_k:]]
                recall_at_k.append(len(gt & set(top_in_cand)) / top_k)
            else:
                recall_at_k.append(0.0)
        
        result = {
            'probes': max_probes,
            'candidates_mean': np.mean(candidate_counts),
            'candidates_std': np.std(candidate_counts),
            'reduction': 1 - np.mean(candidate_counts) / len(embeddings),
            'filter_recall': np.mean(filter_recalls),
            'recall_at_k': np.mean(recall_at_k),
            'time_ms': np.mean(times) * 1000,
        }
        results.append(result)
    
    print(f'\n{label} (band_width={band_width}, {n_bands} bands):')
    print(f'{"Probes":>6} {"Candidates":>12} {"Reduction":>10} {"FilterRecall":>13} {"Recall@10":>10} {"Time(ms)":>10}')
    print('-'*65)
    for r in results:
        print(f'{r["probes"]:>6} {r["candidates_mean"]:>10.0f}±{r["candidates_std"]:>3.0f} '
              f'{r["reduction"]*100:>9.1f}% {r["filter_recall"]*100:>12.1f}% '
              f'{r["recall_at_k"]*100:>9.1f}% {r["time_ms"]:>9.2f}')
    
    return results

In [7]:
print('='*60)
print('DF-LSH Band Filter Evaluation')
print('='*60)

# 英語 PCA - 異なるband_width
for bw in [8, 16, 32]:
    evaluate_band_filter(
        en_embeddings, dflsh_pca_en_codes, dflsh_pca_en_proj,
        band_width=bw, label=f'English PCA'
    )

DF-LSH Band Filter Evaluation



English PCA (band_width=8, 16 bands):
Probes   Candidates  Reduction  FilterRecall  Recall@10   Time(ms)
-----------------------------------------------------------------
     0        659± 96      93.4%         46.8%      46.6%      0.21
     4        803± 97      92.0%         48.7%      48.5%      0.46
     8        950±102      90.5%         52.1%      51.9%      0.52
    16       1270±143      87.3%         68.2%      68.1%      0.63



English PCA (band_width=16, 8 bands):
Probes   Candidates  Reduction  FilterRecall  Recall@10   Time(ms)
-----------------------------------------------------------------
     0          3±  4     100.0%          6.1%       6.0%      0.06
     4          4±  4     100.0%          6.2%       6.1%      0.21
     8          8± 21      99.9%          8.4%       8.4%      0.24
    16          8± 21      99.9%          8.4%       8.4%      0.24



English PCA (band_width=32, 4 bands):
Probes   Candidates  Reduction  FilterRecall  Recall@10   Time(ms)
-----------------------------------------------------------------
     0          0±  2     100.0%          1.4%       1.3%      0.05
     4          0±  2     100.0%          1.5%       1.4%      0.18
     8          0±  2     100.0%          1.5%       1.4%      0.18
    16          0±  2     100.0%          1.5%       1.4%      0.18


In [8]:
# 英語 Random - 比較用
print('\n--- PCA vs Random comparison (band_width=16) ---')
evaluate_band_filter(
    en_embeddings, dflsh_pca_en_codes, dflsh_pca_en_proj,
    band_width=16, label='English PCA'
)
evaluate_band_filter(
    en_embeddings, dflsh_rand_en_codes, dflsh_rand_en_proj,
    band_width=16, label='English Random'
)


--- PCA vs Random comparison (band_width=16) ---



English PCA (band_width=16, 8 bands):
Probes   Candidates  Reduction  FilterRecall  Recall@10   Time(ms)
-----------------------------------------------------------------
     0          3±  4     100.0%          6.1%       6.0%      0.06
     4          4±  4     100.0%          6.2%       6.1%      0.21
     8          8± 21      99.9%          8.4%       8.4%      0.24
    16          8± 21      99.9%          8.4%       8.4%      0.24



English Random (band_width=16, 8 bands):
Probes   Candidates  Reduction  FilterRecall  Recall@10   Time(ms)
-----------------------------------------------------------------
     0          8± 50      99.9%          3.9%       3.8%      0.06
     4          9± 50      99.9%          4.2%       4.1%      0.21
     8         10± 50      99.9%          5.0%       4.9%      0.24
    16         10± 50      99.9%          5.0%       4.9%      0.24


[{'probes': 0,
  'candidates_mean': np.float64(7.56),
  'candidates_std': np.float64(49.862675419596165),
  'reduction': np.float64(0.999244),
  'filter_recall': np.float64(0.039),
  'recall_at_k': np.float64(0.038),
  'time_ms': np.float64(0.06251335144042969)},
 {'probes': 4,
  'candidates_mean': np.float64(8.68),
  'candidates_std': np.float64(49.870408059289026),
  'reduction': np.float64(0.999132),
  'filter_recall': np.float64(0.042),
  'recall_at_k': np.float64(0.040999999999999995),
  'time_ms': np.float64(0.21416902542114258)},
 {'probes': 8,
  'candidates_mean': np.float64(10.02),
  'candidates_std': np.float64(50.40317053519551),
  'reduction': np.float64(0.998998),
  'filter_recall': np.float64(0.05),
  'recall_at_k': np.float64(0.049),
  'time_ms': np.float64(0.24455547332763672)},
 {'probes': 16,
  'candidates_mean': np.float64(10.02),
  'candidates_std': np.float64(50.40317053519551),
  'reduction': np.float64(0.998998),
  'filter_recall': np.float64(0.05),
  'recall_at_

In [9]:
# 日本語 PCA
print('\n--- Japanese PCA (band_width=16) ---')
evaluate_band_filter(
    ja_embeddings, dflsh_pca_ja_codes, dflsh_pca_ja_proj,
    band_width=16, label='Japanese PCA'
)


--- Japanese PCA (band_width=16) ---



Japanese PCA (band_width=16, 8 bands):
Probes   Candidates  Reduction  FilterRecall  Recall@10   Time(ms)
-----------------------------------------------------------------
     0          4±  8     100.0%          3.2%       3.2%      0.06
     4          5±  8     100.0%          3.2%       3.2%      0.21
     8          7± 11      99.9%          5.3%       5.3%      0.24
    16          7± 11      99.9%          5.3%       5.3%      0.24


[{'probes': 0,
  'candidates_mean': np.float64(4.15),
  'candidates_std': np.float64(8.362266439189796),
  'reduction': np.float64(0.999585),
  'filter_recall': np.float64(0.032),
  'recall_at_k': np.float64(0.032),
  'time_ms': np.float64(0.06066083908081055)},
 {'probes': 4,
  'candidates_mean': np.float64(4.84),
  'candidates_std': np.float64(8.297855144553921),
  'reduction': np.float64(0.999516),
  'filter_recall': np.float64(0.032),
  'recall_at_k': np.float64(0.032),
  'time_ms': np.float64(0.2105998992919922)},
 {'probes': 8,
  'candidates_mean': np.float64(6.98),
  'candidates_std': np.float64(11.067953740416518),
  'reduction': np.float64(0.999302),
  'filter_recall': np.float64(0.05299999999999999),
  'recall_at_k': np.float64(0.05299999999999999),
  'time_ms': np.float64(0.24135351181030273)},
 {'probes': 16,
  'candidates_mean': np.float64(6.98),
  'candidates_std': np.float64(11.067953740416518),
  'reduction': np.float64(0.999302),
  'filter_recall': np.float64(0.0529999

## 5. ITQ-LSH ベースライン比較

In [10]:
def evaluate_itq_baseline(embeddings, hashes, label, n_queries=100, top_k=10, 
                          candidate_limits=[100, 500, 1000]):
    """ITQ Hamming距離ベースのRecall@10"""
    rng = np.random.default_rng(42)
    query_indices = rng.choice(len(embeddings), n_queries, replace=False)
    
    results = {}
    
    for limit in candidate_limits:
        recalls = []
        times = []
        
        for qi in query_indices:
            cos_sims = cosine_similarity(embeddings[qi:qi+1], embeddings)[0]
            cos_sims[qi] = -1
            gt = set(np.argsort(cos_sims)[-top_k:])
            
            start = time.time()
            h_dists = hamming_distance_batch(hashes[qi], hashes).astype(float)
            h_dists[qi] = np.inf
            candidates = set(np.argsort(h_dists)[:limit])
            
            # Cosine rerank
            cand_arr = np.array(list(candidates))
            cand_cos = cosine_similarity(embeddings[qi:qi+1], embeddings[cand_arr])[0]
            top_in_cand = cand_arr[np.argsort(cand_cos)[-top_k:]]
            recalls.append(len(gt & set(top_in_cand)) / top_k)
            times.append(time.time() - start)
        
        results[limit] = {
            'recall': np.mean(recalls),
            'time_ms': np.mean(times) * 1000
        }
    
    print(f'\n{label} - ITQ Baseline:')
    print(f'{"Candidates":>12} {"Recall@10":>10} {"Time(ms)":>10}')
    print('-'*35)
    for limit in candidate_limits:
        r = results[limit]
        print(f'{limit:>12} {r["recall"]*100:>9.1f}% {r["time_ms"]:>9.2f}')
    
    return results

print('='*60)
print('ITQ-LSH Baseline (Hamming distance sort + Cosine rerank)')
print('='*60)

itq_en_results = evaluate_itq_baseline(en_embeddings, en_hashes, 'E5-base English')
itq_ja_results = evaluate_itq_baseline(ja_embeddings, ja_hashes, 'E5-base Japanese')

ITQ-LSH Baseline (Hamming distance sort + Cosine rerank)



E5-base English - ITQ Baseline:
  Candidates  Recall@10   Time(ms)
-----------------------------------
         100      61.5%      1.59
         500      83.8%      2.41
        1000      91.4%      3.51



E5-base Japanese - ITQ Baseline:
  Candidates  Recall@10   Time(ms)
-----------------------------------
         100      83.1%      1.61
         500      98.1%      2.41
        1000      99.0%      3.48


## 6. ITQバンドインデックス比較

DF-LSHのバンドインデックス手法をITQハッシュにも適用して比較

In [11]:
# ITQハッシュでもバンドインデックスを構築
print('='*60)
print('ITQ Hash with Band Indexing (for comparison)')
print('='*60)

# ITQの確信度情報を取得
_, itq_en_proj = itq.transform_with_confidence(en_embeddings)
_, itq_ja_proj = itq.transform_with_confidence(ja_embeddings)

evaluate_band_filter(
    en_embeddings, en_hashes, itq_en_proj,
    band_width=16, label='English ITQ+Band'
)

evaluate_band_filter(
    ja_embeddings, ja_hashes, itq_ja_proj,
    band_width=16, label='Japanese ITQ+Band'
)

ITQ Hash with Band Indexing (for comparison)



English ITQ+Band (band_width=16, 8 bands):
Probes   Candidates  Reduction  FilterRecall  Recall@10   Time(ms)
-----------------------------------------------------------------
     0         33± 57      99.7%          7.9%       7.8%      0.07
     4         40± 68      99.6%          9.0%       8.9%      0.23
     8         56± 75      99.4%         11.7%      11.6%      0.26
    16         56± 75      99.4%         11.7%      11.6%      0.26



Japanese ITQ+Band (band_width=16, 8 bands):
Probes   Candidates  Reduction  FilterRecall  Recall@10   Time(ms)
-----------------------------------------------------------------
     0          5±  8     100.0%          3.8%       3.8%      0.06
     4          6±  9      99.9%          5.4%       5.4%      0.21
     8          8± 12      99.9%          7.3%       7.3%      0.25
    16          8± 12      99.9%          7.3%       7.3%      0.24


[{'probes': 0,
  'candidates_mean': np.float64(4.54),
  'candidates_std': np.float64(8.107305347647884),
  'reduction': np.float64(0.999546),
  'filter_recall': np.float64(0.038000000000000006),
  'recall_at_k': np.float64(0.038000000000000006),
  'time_ms': np.float64(0.06137847900390625)},
 {'probes': 4,
  'candidates_mean': np.float64(5.96),
  'candidates_std': np.float64(9.420106156514374),
  'reduction': np.float64(0.999404),
  'filter_recall': np.float64(0.05399999999999999),
  'recall_at_k': np.float64(0.05399999999999999),
  'time_ms': np.float64(0.21335840225219727)},
 {'probes': 8,
  'candidates_mean': np.float64(8.05),
  'candidates_std': np.float64(11.633894446830778),
  'reduction': np.float64(0.999195),
  'filter_recall': np.float64(0.07300000000000001),
  'recall_at_k': np.float64(0.07300000000000001),
  'time_ms': np.float64(0.24584293365478518)},
 {'probes': 16,
  'candidates_mean': np.float64(8.05),
  'candidates_std': np.float64(11.633894446830778),
  'reduction': np

## 7. MiniLM検証

In [12]:
print('='*60)
print('MiniLM Verification (English only)')
print('='*60)

# MiniLMのDF-LSH
dflsh_minilm = DFLSH(n_projections=128, band_width=16, seed=42)
dflsh_minilm.fit(minilm_embeddings, method='pca')
minilm_dflsh_codes, minilm_dflsh_proj = dflsh_minilm.transform_with_confidence(minilm_embeddings)

print('\n--- Code Quality ---')
evaluate_code_quality(minilm_embeddings, minilm_hashes, 'ITQ-LSH')
evaluate_code_quality(minilm_embeddings, minilm_dflsh_codes, 'DF-LSH (PCA)')

print('\n--- Band Filter ---')
evaluate_band_filter(
    minilm_embeddings, minilm_dflsh_codes, minilm_dflsh_proj,
    band_width=16, label='MiniLM DF-LSH(PCA)'
)

# MiniLM ITQの確信度
_, minilm_itq_proj = itq_minilm.transform_with_confidence(minilm_embeddings)
evaluate_band_filter(
    minilm_embeddings, minilm_hashes, minilm_itq_proj,
    band_width=16, label='MiniLM ITQ+Band'
)

print('\n--- ITQ Baseline ---')
evaluate_itq_baseline(minilm_embeddings, minilm_hashes, 'MiniLM')

MiniLM Verification (English only)
DFLSH fit (PCA): dim=384, projections=128, explained_variance=79.61%

--- Code Quality ---


ITQ-LSH: Spearman=-0.6897 (p=0.00e+00)


DF-LSH (PCA): Spearman=-0.4626 (p=0.00e+00)

--- Band Filter ---



MiniLM DF-LSH(PCA) (band_width=16, 8 bands):
Probes   Candidates  Reduction  FilterRecall  Recall@10   Time(ms)
-----------------------------------------------------------------
     0          2±  2     100.0%          2.4%       2.4%      0.06
     4          3±  2     100.0%          2.5%       2.5%      0.21
     8          4±  4     100.0%          4.5%       4.5%      0.24
    16          4±  4     100.0%          4.5%       4.5%      0.25



MiniLM ITQ+Band (band_width=16, 8 bands):
Probes   Candidates  Reduction  FilterRecall  Recall@10   Time(ms)
-----------------------------------------------------------------
     0          2±  3     100.0%          3.2%       3.2%      0.06
     4          3±  3     100.0%          4.0%       4.0%      0.22
     8          5±  4     100.0%          5.5%       5.5%      0.24
    16          5±  4     100.0%          5.5%       5.5%      0.26

--- ITQ Baseline ---



MiniLM - ITQ Baseline:
  Candidates  Recall@10   Time(ms)
-----------------------------------
         100      88.3%      1.66
         500      98.2%      2.48
        1000      99.3%      3.47


{100: {'recall': np.float64(0.8830000000000001),
  'time_ms': np.float64(1.6570711135864258)},
 500: {'recall': np.float64(0.982), 'time_ms': np.float64(2.477412223815918)},
 1000: {'recall': np.float64(0.9930000000000001),
  'time_ms': np.float64(3.46860408782959)}}

## 8. 総合比較サマリー

In [13]:
print('='*80)
print('DF-LSH vs ITQ-LSH Head-to-Head Comparison Summary')
print('='*80)

print('\n【バイナリコード品質 (Hamming-Cosine Spearman)】')
print(f'{"":<20} {"ITQ-LSH":>10} {"DF-LSH(PCA)":>12} {"DF-LSH(Rand)":>13}')
print('-'*60)
print(f'{"English E5-base":<20} {corr_itq_en:>10.4f} {corr_pca_en:>12.4f} {corr_rand_en:>13.4f}')
print(f'{"Japanese E5-base":<20} {corr_itq_ja:>10.4f} {corr_pca_ja:>12.4f} {"---":>13}')

print('\n【結論】')
if abs(corr_itq_en) > abs(corr_pca_en):
    print('ITQ-LSHのバイナリコード品質がDF-LSH(PCA)を上回る')
    print('→ ITQ回転行列の最適化が量子化品質に寄与')
else:
    print('DF-LSH(PCA)がITQ-LSHに匹敵/上回るバイナリコード品質')

print('\nDF-LSHのバンドインデックス方式は、候補選択の高速化に貢献する可能性がある。')
print('ただし、ITQの回転最適化なしでは、Hamming距離とCosine類似度の相関が低下する。')
print('→ 実験82・83でITQハッシュ上でのバンドインデックス/multi-probeを評価する。')

DF-LSH vs ITQ-LSH Head-to-Head Comparison Summary

【バイナリコード品質 (Hamming-Cosine Spearman)】
                        ITQ-LSH  DF-LSH(PCA)  DF-LSH(Rand)
------------------------------------------------------------
English E5-base         -0.4717      -0.2278       -0.2505
Japanese E5-base        -0.5283      -0.2771           ---

【結論】
ITQ-LSHのバイナリコード品質がDF-LSH(PCA)を上回る
→ ITQ回転行列の最適化が量子化品質に寄与

DF-LSHのバンドインデックス方式は、候補選択の高速化に貢献する可能性がある。
ただし、ITQの回転最適化なしでは、Hamming距離とCosine類似度の相関が低下する。
→ 実験82・83でITQハッシュ上でのバンドインデックス/multi-probeを評価する。
